In [162]:
import torch

In [163]:
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [164]:
cancer = load_breast_cancer()

In [165]:
X = cancer.data
y = cancer.target

In [166]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

In [167]:
X_train = torch.tensor(X_train, dtype = torch.float32)
X_test = torch.tensor(X_test, dtype = torch.float32)

y_train = torch.tensor(y_train, dtype = torch.float32).unsqueeze(-1)
y_test = torch.tensor(y_test, dtype = torch.float32).unsqueeze(-1)

In [168]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([455, 30]),
 torch.Size([114, 30]),
 torch.Size([455, 1]),
 torch.Size([114, 1]))

In [169]:
tr_data = TensorDataset(X_train, y_train)
val_data = TensorDataset(X_test, y_test)

In [295]:
tr_loader = DataLoader(tr_data, batch_size = 16, shuffle = True)
val_loader = DataLoader(val_data, batch_size = 1, shuffle = False)

In [312]:
class Binary_Net(nn.Module) :
  def __init__(self) :
    super(Binary_Net, self).__init__()

    self.layer_1 = nn.Linear(30, 32)
    self.layer_2 = nn.Linear(32, 32)
    self.relu_1 = nn.ReLU()
    self.layer_3 = nn.Linear(32, 16)
    self.layer_4 = nn.Linear(16, 8)
    self.relu_2 = nn.ReLU()
    self.layer_out = nn.Linear(8, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, input) :
    x = self.layer_1(input)
    x = self.layer_2(x)
    x = self.relu_1(x)
    x = self.layer_3(x)
    x = self.layer_4(x)
    x = self.relu_2(x)
    x = self.layer_out(x)
    x = self.sigmoid(x)

    return x

In [313]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [314]:
model = Binary_Net()

In [315]:
model.to(device)

Binary_Net(
  (layer_1): Linear(in_features=30, out_features=32, bias=True)
  (layer_2): Linear(in_features=32, out_features=32, bias=True)
  (relu_1): ReLU()
  (layer_3): Linear(in_features=32, out_features=16, bias=True)
  (layer_4): Linear(in_features=16, out_features=8, bias=True)
  (relu_2): ReLU()
  (layer_out): Linear(in_features=8, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [316]:
def eval_model(model, data_loader) :
  model.eval()

  ys = []
  y_preds = []

  for x, y in data_loader :

    x = x.to(device)
    y = y.to(device)

    with torch.no_grad() :
      _, y_pred = model(x).max(1)
    ys.append(y)
    y_preds.append(y_pred)

  ys = torch.cat(ys)
  y_preds = torch.cat(y_preds)
  
  acc = (ys == y_preds)[:, 0].float().sum() / len(data_loader)

  return acc.item()

In [331]:
def train_model(model, train_loader, val_loader, epochs) :

  tr_losses = []
  val_acc = []

  optimizer = optim.Adam
  optimizer = optimizer(model.parameters(), lr = 0.005)
  loss_fn = nn.BCEWithLogitsLoss()

  for epoch in range(epochs) :
    print(f"Epoch {epoch + 1}")

    epoch_loss = 0
    model.train()
    tr_acc = 0

    for i, (xx, yy) in enumerate(train_loader) :

      xx = xx.to(device)
      yy = yy.to(device)

      Model = model(xx)

      loss = loss_fn(Model, yy)

      optimizer.zero_grad()

      loss.backward()

      optimizer.step()

      epoch_loss += loss.item()

      _, y_pred = Model.max(1)
      tr_acc += (yy == y_pred)[:, 0].float().sum().item() / len(xx)
    print(f"Accuracy of Train Data = {tr_acc / len(train_loader)}\n")
    tr_losses.append(loss / i)

  val_acc = eval_model(model, val_loader)
  print(f"\nAccuracy of Validation Data = {val_acc}")

In [332]:
train_model(model, tr_loader, val_loader, epochs = 10)

Epoch 1
Accuracy of Train Data = 0.374692118226601

Epoch 2
Accuracy of Train Data = 0.3691502463054187

Epoch 3
Accuracy of Train Data = 0.3719211822660099

Epoch 4
Accuracy of Train Data = 0.374692118226601

Epoch 5
Accuracy of Train Data = 0.3691502463054187

Epoch 6
Accuracy of Train Data = 0.374692118226601

Epoch 7
Accuracy of Train Data = 0.374692118226601

Epoch 8
Accuracy of Train Data = 0.374692118226601

Epoch 9
Accuracy of Train Data = 0.3719211822660099

Epoch 10
Accuracy of Train Data = 0.3719211822660099


Accuracy of Validation Data = 0.3684210479259491
